In [ ]:
# Import necessary libraries
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func

# Create an engine to connect to the database
database_path = "C:/Users/messa/SQLalchemy-challenge/Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

# Reflect the database tables into classes
Base = automap_base()
Base.prepare(engine, reflect=True)

# Save references to the classes
Station = Base.classes.station
Measurement = Base.classes.measurement

# Create a session to interact with the database
session = Session(engine)


In [ ]:
from sqlalchemy import func

most_recent_date = session.query(func.max(Measurement.date)).scalar()
print("Most recent date:", most_recent_date)


In [ ]:
from datetime import datetime, timedelta

# Calculate the date 12 months ago from the most recent date
one_year_ago = datetime.strptime(most_recent_date, "%Y-%m-%d") - timedelta(days=365)

# Query the precipitation data for the last 12 months
precipitation_data = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= one_year_ago).all()


In [ ]:
# Load data into a DataFrame
precipitation_df = pd.DataFrame(precipitation_data, columns=["date", "prcp"])

# Sort DataFrame values by date
precipitation_df.sort_values("date", inplace=True)

# Plot the precipitation data
plt.figure(figsize=(10, 6))
plt.bar(precipitation_df["date"], precipitation_df["prcp"], label="precipitation")
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")
plt.title("Precipitation in the Last 12 Months")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
summary_stats = precipitation_df.describe()
print(summary_stats)


In [ ]:
total_stations = session.query(Station.station).count()
print("Total number of stations:", total_stations)


In [ ]:
from sqlalchemy import func

# Query to list stations and observation counts in descending order
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()

# Extract the most active station and its observation count
most_active_station = most_active_stations[0][0]
most_active_count = most_active_stations[0][1]

print("Most active station:", most_active_station)
print("Observation count:", most_active_count)


In [ ]:
lowest_temp = session.query(func.min(Measurement.tobs)).filter(Measurement.station == most_active_station).scalar()
highest_temp = session.query(func.max(Measurement.tobs)).filter(Measurement.station == most_active_station).scalar()
avg_temp = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == most_active_station).scalar()

print("Lowest temperature:", lowest_temp)
print("Highest temperature:", highest_temp)
print("Average temperature:", avg_temp)


In [ ]:
from datetime import datetime, timedelta

# Calculate the date 12 months ago from the most recent date
one_year_ago = datetime.strptime(most_recent_date, "%Y-%m-%d") - timedelta(days=365)

# Query TOBS data for the most-active station for the last 12 months
tobs_data = session.query(Measurement.tobs).\
    filter(Measurement.station == most_active_station, Measurement.date >= one_year_ago).all()

# Convert to a list for plotting
tobs_list = [temp[0] for temp in tobs_data]

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(tobs_list, bins=12, label="TOBS")
plt.xlabel("Temperature (°F)")
plt.ylabel("Frequency")
plt.title("Temperature Observation Distribution")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
    session.close()


In [ ]:
# Import necessary libraries
from flask import Flask, jsonify

# Create a Flask app
app = Flask(__name__)

# Define the home route to list available routes
@app.route("/")
def home():
    return (
        "Available Routes:<br/>"
        "/api/v1.0/precipitation<br/>"
        "/api/v1.0/stations<br/>"
        "/api/v1.0/tobs<br/>"
        "/api/v1.0/<start><br/>"
        "/api/v1.0/<start>/<end><br/>"
    )



In [ ]:
# Import necessary libraries
from flask import Flask, jsonify

# Create a Flask app
app = Flask(__name__)

# Define the home route to list available routes
@app.route("/")
def home():
    return (
        "Welcome to the Climate Analysis API!<br/>"
        "Available Routes:<br/>"
        "/api/v1.0/precipitation<br/>"
        "/api/v1.0/stations<br/>"
        "/api/v1.0/tobs<br/>"
        "/api/v1.0/&lt;start&gt;<br/>"
        "/api/v1.0/&lt;start&gt;/&lt;end&gt;<br/>"
    )



In [ ]:
# Define the /api/v1.0/precipitation route
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Query to retrieve the last 12 months of precipitation data
    precipitation_data = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= one_year_ago).all()

    # Convert the query results to a dictionary using date as the key and prcp as the value
    precipitation_dict = {date: prcp for date, prcp in precipitation_data}

    # Return the JSON representation of the dictionary
    return jsonify(precipitation_dict)



In [ ]:
# Define the /api/v1.0/stations route
@app.route("/api/v1.0/stations")
def stations():
    # Query to retrieve the list of stations
    stations_list = session.query(Station.station).all()

    # Flatten the list of tuples into a single list
    stations_flat = [station[0] for station in stations_list]

    # Return a JSON list of station names
    return jsonify(stations_flat)


In [ ]:
# Define the /api/v1.0/tobs route
@app.route("/api/v1.0/tobs")
def tobs():
    # Query to retrieve temperature observations for the most-active station for the previous year
    tobs_data = session.query(Measurement.date, Measurement.tobs).\
        filter(Measurement.station == most_active_station, Measurement.date >= one_year_ago).all()

    # Convert the query results to a list of temperature values
    tobs_list = [tobs for date, tobs in tobs_data]

    # Return a JSON list of temperature observations
    return jsonify(tobs_list)
    

In [ ]:
# Define the /api/v1.0/<start> and /api/v1.0/<start>/<end> routes
@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def temperature_range(start, end=None):
    # Query to calculate min, avg, and max temperatures for the specified range
    if end is None:
        temperature_data = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
            filter(Measurement.date >= start).all()
    else:
        temperature_data = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
            filter(Measurement.date >= start, Measurement.date <= end).all()

    # Create a dictionary with keys for each statistic
    temperature_dict = {
        "min_temperature": temperature_data[0][0],
        "avg_temperature": temperature_data[0][1],
        "max_temperature": temperature_data[0][2]
    }

    # Return a JSON representation of the dictionary
    return jsonify(temperature_dict)


In [ ]:
# Run the app
if __name__ == "__main__":
    app.run(debug=True)